In [109]:
from dataclasses import dataclass
from datetime import date, timedelta

from prettytable import PrettyTable, TableStyle

from lib.utils.interest import annual_to_daily_interest_rate
from lib.utils.date import days_in_year

# Daily Interest Calculations

Interest payments on bank accounts are often applied annually or quarterly.

For example, the ABN AMRO Capital Savings Account applies interest annually, whereas their Deposit Accounts
apply interest quarterly.

In both cases, however, the rate advertised is the effective annual rate.

For annual payments, this is simple to calculate as it can be prorated over the number of days spent at
each balance level.

The following examples have bene taken from the ABN AMRO site at https://www.abnamro.nl/en/personal/savings/interest-rates/when-and-how-often-do-you-receive-interest.html

## Annual interest payments

For example, on 31 December, the balance in your savings account was € 5,000.
You received € 2,000 in holiday pay that you put in your savings account on 25 May.
On 15 August, you went on holiday and withdrew € 1,000 from the account.
The savings rate in this sample calculation is 1.50% and does not change during the year.
The interest is calculated as follows:

In [110]:
INTEREST_RATE = 0.015

@dataclass(frozen=True)
class Period:
    start: date
    end: date
    balance: float

ANNUAL_PERIODS = [Period(start=date(2025, 12, 31),
                         end=date(2026, 5, 24),
                         balance=5000),
                  Period(start=date(2026, 5, 25),
                  end=date(2026, 8, 14),
                  balance=7000),
                  Period(start=date(2026, 8, 15),
                  end=date(2026, 12, 30),
                  balance=6000)]

annual_table = PrettyTable()
annual_table.field_names = ['Period', 'Balance', 'Days', 'Interest']
interest_payments = []
for period in ANNUAL_PERIODS:
    days = (period.end - period.start).days + 1
    interest_payment = INTEREST_RATE * period.balance * (days/365)
    annual_table.add_row([f'{period.start} to {period.end}',
                          period.balance,
                          days,
                          interest_payment])
    interest_payments.append(interest_payment)
ORIGINAL_ANNUAL_INTEREST_PAYMENT = sum(interest_payments)
annual_table.add_row(['', '', 'Total', ORIGINAL_ANNUAL_INTEREST_PAYMENT])
annual_table.set_style(TableStyle.SINGLE_BORDER)
annual_table.float_format = '.2'
annual_table

Period,Balance,Days,Interest
2025-12-31 to 2026-05-24,5000,145,29.79
2026-05-25 to 2026-08-14,7000,82,23.59
2026-08-15 to 2026-12-30,6000,138,34.03
,,Total,87.41


Note that here we have assumed that there are 365 days in a year. Although it is not stated on the ABN AMRO
site, I assume that in leap years they use the same annual rate and spread it over 366 days. After all, that would
be most in their favor, and they don't state otherwise.

## Quarterly interest payments

As far as I can tell, the advertised rates for quarterly payments are also the annual effective rates. This
means that the actual rate applied each quarter will be the nominal rate that when compounded will result in
the annual effective rate.

The nominal rate can be calculated using the following formula:

```
nominal_rate =
    number_of_periods * [ { (1 + effective_rate) ^ (1 / number_of_periods) } - 1 ]
```

In [111]:
QUARTERLY_EFFECTIVE_RATES = [0.015, 0.013, 0.011, 0.002]
NUMBER_OF_PERIODS = 4

quarterly_rates_table = PrettyTable()
quarterly_rates_table.field_names = ['Nominal rate (once a quarter)', 'Effective rate']
for effective_rate in QUARTERLY_EFFECTIVE_RATES:
    nominal_rate = NUMBER_OF_PERIODS * (pow(1 + effective_rate, 1 / NUMBER_OF_PERIODS) - 1)
    quarterly_rates_table.add_row([f'{(nominal_rate * 100):.3f}%', f'{(effective_rate * 100):.2f}%'])
quarterly_rates_table.set_style(TableStyle.SINGLE_BORDER)
quarterly_rates_table

Nominal rate (once a quarter),Effective rate
1.492%,1.50%
1.294%,1.30%
1.095%,1.10%
0.200%,0.20%


Note that this is still an annual rate, but as it is applied four times each year it compounds back to the
effective rate. This does mean, however, that the formula to calculate the interest for each period remains
the same with the number of days divided by the number of days in the year (not the quarter).

We can show this by comparing the interest applied annually or quarterly, over a single year on € 10,000, at an effective
rate of 1.5%.

In [112]:
OPENING_BALANCE = 10_000
EFFECTIVE_RATE = 0.015
DAYS_IN_YEAR = 365
PERIOD_COUNT = 4
DAYS_IN_PERIOD = DAYS_IN_YEAR / PERIOD_COUNT
NOMINAL_RATE = PERIOD_COUNT * (pow(1 + EFFECTIVE_RATE, 1 / PERIOD_COUNT) - 1)

annual_interest_payment = OPENING_BALANCE * EFFECTIVE_RATE

print(f'Annual interest payment: {annual_interest_payment:.6f}')

quarterly_interest_payments = []
current_balance = OPENING_BALANCE
for days in [DAYS_IN_PERIOD] * PERIOD_COUNT:
    interest_payment = current_balance * NOMINAL_RATE * (days / DAYS_IN_YEAR)
    quarterly_interest_payments.append(interest_payment)
    current_balance += interest_payment
total_quarterly_interest_payment = sum(quarterly_interest_payments)

print(f'Total quarterly interest: {total_quarterly_interest_payment:.6f}')

Annual interest payment: 150.000000
Total quarterly interest: 150.000000


## Daily interest calculations

In our simulations, we will actually increment the states daily. This actually simplifies the interest
calculations for us as the number of periods will be the number of days in the current year.

This also simplifies the application of leap years as a day is either in a leap year or not.

Remember that the annual nominal rate was calculated by multiplying by the period count.

Also remember that the periodic interest payment is calculated by dividing by the days in the year.

For daily calculations, the period count and the days in the year are equal!

Also for daily calculations, the number of days in each period becomes 1 which removes another multiplier
from the formula.

Thus, the daily calculation becomes:

```
daily_rate = [ (1 + effective_rate) ^ (1 / days_in_year) ] - 1
```

This formula is captured in the `lib.utils.interest.annual_to_daily_interest_rate` function. A function to
get the number of days in a year is also provided in `lib.utils.date.days_in_year`.

This rate **compounded** every day over the year should result in the same effective rate. Using the
example from above we can demonstrate this:

In [113]:
CURRENT_DATE = date.today()
DAILY_RATE = annual_to_daily_interest_rate(EFFECTIVE_RATE, CURRENT_DATE)
interest_accrued = 0
for day in range(days_in_year(CURRENT_DATE.year)):
    interest_accrued += (OPENING_BALANCE + interest_accrued) * DAILY_RATE
print(f'Interest accrued: {interest_accrued:.6f}')

Interest accrued: 150.000000


You can see that in this calculation, the interest accrued is added to the balance for each calculation
to ensure the correct compounding effects.

You can also see that on any arbitrary day the interest accrued could be added to the balance and zeroed
out without impacting the calculation. To see this, we can arbitrarily pay out interest every 10th day:

In [114]:
CURRENT_DATE = date.today()
DAILY_RATE = annual_to_daily_interest_rate(EFFECTIVE_RATE, CURRENT_DATE)
interest_accrued = 0
current_balance = OPENING_BALANCE
for day in range(days_in_year(CURRENT_DATE.year)):
    if day % 10 == 0:
        current_balance += interest_accrued
        interest_accrued = 0
    interest_accrued += (current_balance + interest_accrued) * DAILY_RATE
print(f'Interest paid or accrued: {current_balance - OPENING_BALANCE + interest_accrued:.6f}')

Interest paid or accrued: 150.000000


This method will also provide the correct calculation in the face of daily deposits and withdrawals as we
no longer have to consider the number of days at each balance. We can demonstrate this by calculating the
interest accrued for our original annual example:

In [115]:
interest_accrued = 0
number_of_days = 0
for period in ANNUAL_PERIODS:
    days = (period.end - period.start).days + 1
    number_of_days += days
    for day in range(days):
        current_date = period.start + timedelta(days=day)
        daily_rate = annual_to_daily_interest_rate(INTEREST_RATE, current_date)
        interest_accrued += (daily_rate * (period.balance + interest_accrued))
print(f'Number of days: {number_of_days}')
print(f'Interest rate: {INTEREST_RATE * 100:.2f}%')
print(f'Daily interest accrued: {interest_accrued:.6f}')
print(f'Original annual interest payment: {ORIGINAL_ANNUAL_INTEREST_PAYMENT:.6f}')

Number of days: 365
Interest rate: 1.50%
Daily interest accrued: 87.383697
Original annual interest payment: 87.410959
